## 1. Create DataFrames

In [1]:
import os
import pandas as pd
import zipfile

# Base directory containing the subdirectories with .pkl.zip files
base_directory = "climbing-data-main"

# Dictionary to store DataFrames with their names
dataframes = {}

# Walk through the directory structure
for root, _, files in os.walk(base_directory):
    for filename in files:
        if filename.endswith(".pkl.zip"):  # Check for .pkl.zip files
            file_path = os.path.join(root, filename)
            
            # Extract the base name for the DataFrame (remove .pkl.zip)
            df_name = os.path.splitext(os.path.splitext(filename)[0])[0]
            
            # Open the .pkl.zip file and load the pickle file into a DataFrame
            with zipfile.ZipFile(file_path, 'r') as z:
                with z.open(z.namelist()[0]) as f:
                    dataframes[df_name] = pd.read_pickle(f)

# Assign DataFrames dynamically as variables in globals()
for df_name, df in dataframes.items():
    globals()[df_name] = df
    print(f"Loaded DataFrame: {df_name} with shape {df.shape}")



Loaded DataFrame: Sector_Addresses with shape (14031, 4)
Loaded DataFrame: RouteQualityData with shape (96722, 19)
Loaded DataFrame: Curated_OpenBetaAug2020_RytherAnderson with shape (168910, 17)
Loaded DataFrame: All_Ratings with shape (1893162, 6)
Loaded DataFrame: CuratedWithRatings_OpenBetaAug2020_RytherAnderson with shape (96963, 18)


In [2]:
# Iterate over the dynamically loaded DataFrames and print their column names
for df_name, df in dataframes.items():
    print(f"DataFrame: {df_name}")
    print("Columns:")
    # print(df.columns.tolist())
    print(df.columns)
    print("-" * 50)  # Separator for readability


DataFrame: Sector_Addresses
Columns:
Index(['parent_sector', 'sector_ID', 'parent_loc', 'sector_address'], dtype='object')
--------------------------------------------------
DataFrame: RouteQualityData
Columns:
Index(['route_name', 'route_ID', 'type_string', 'sector_ID', 'parent_sector',
       'parent_loc', 'num_votes', 'adjusted_num_votes', 'mean_rating',
       'median_rating', 'mode_rating', 'RQI_mean', 'RQI_median', 'ARQI_mean',
       'ARQI_median', 'nopm_YDS', 'YDS_rank', 'safety', 'state'],
      dtype='object')
--------------------------------------------------
DataFrame: Curated_OpenBetaAug2020_RytherAnderson
Columns:
Index(['route_name', 'parent_sector', 'route_ID', 'sector_ID', 'type_string',
       'fa', 'YDS', 'Vermin', 'nopm_YDS', 'nopm_Vermin', 'YDS_rank',
       'Vermin_rank', 'safety', 'parent_loc', 'description', 'location',
       'protection'],
      dtype='object')
--------------------------------------------------
DataFrame: All_Ratings
Columns:
Index(['users', '

## 2. Filtering Columns

In [3]:
# Convert route_ID to string for all DataFrames
Curated_OpenBetaAug2020_RytherAnderson['route_ID'] = Curated_OpenBetaAug2020_RytherAnderson['route_ID'].astype(str)
RouteQualityData['route_ID'] = RouteQualityData['route_ID'].astype(str)
All_Ratings['route_ID'] = All_Ratings['route_id'].astype(str)

In [4]:
# calculate the mean rating for each route
ratings_agg = All_Ratings.groupby('route_ID')['ratings'].mean().reset_index()

In [5]:
# seperate log and lat from the location column
Sector_Addresses[['Latitude', 'Longitude']] = pd.DataFrame(Sector_Addresses['parent_loc'].tolist(), index=Sector_Addresses.index)

# Convert latitude and longitude to float for further processing
Sector_Addresses['Latitude'] = Sector_Addresses['Latitude'].astype(float)
Sector_Addresses['Longitude'] = Sector_Addresses['Longitude'].astype(float)

In [6]:
# Filter useful columns from each DataFrame
Sector_Addresses_filtered = Sector_Addresses[['sector_ID', 'parent_sector', 'Latitude', 'Longitude']]
RouteQualityData_filtered = RouteQualityData[['route_ID', 'num_votes', 'state', 'RQI_mean']]
Curated_OpenBetaAug2020_filtered = Curated_OpenBetaAug2020_RytherAnderson[['route_ID', 'route_name', 'type_string', 'sector_ID', 'YDS', 'description', 'protection']]
All_Ratings_filtered = ratings_agg[['route_ID', 'ratings']]

## 3. Merge DataFrames

In [7]:
# Step 1: Merge Curated_OpenBetaAug2020 with RouteQualityData
merged_df = pd.merge(
    Curated_OpenBetaAug2020_filtered,
    RouteQualityData_filtered,
    on="route_ID",
    how="inner"
)

# Step 2: Merge with Sector_Addresses
merged_df = pd.merge(
    merged_df,
    Sector_Addresses_filtered,
    on="sector_ID",
    how="inner"
)

# Step 3: Merge with All_Ratings
final_merged_df = pd.merge(
    merged_df,
    All_Ratings_filtered,
    on="route_ID",
    how="left"
)


In [8]:
# Rename columns for better readability
cleaned_df = final_merged_df.rename(columns={
    'route_ID': 'Route_ID',
    'route_name': 'Route_Name',
    'sector_ID': 'Sector_ID',
    'YDS': 'Grade(YDS)',
    'description': 'Route_Description',
    'protection': 'Protection_Type',
    'num_votes': 'Popularity_Votes',
    'state': 'State',
    'RQI_mean': 'Route_Quality_Mean',
    'parent_sector': 'Region/Area',
    'ratings': 'Average_Rating',
    'type_string': 'Route_Type'
})

# Convert columns to correct data types
cleaned_df = cleaned_df.astype({
    'Route_ID': 'str',
    'Route_Name': 'str',
    'Sector_ID': 'str',
    'Grade(YDS)': 'str',
    'Route_Description': 'str',
    'Protection_Type': 'str',
    'State': 'str',
    'Region/Area': 'str',
    "Popularity_Votes": 'int'
})

## 4. Data Cleaning

In [9]:
# Check unique values for specific columns
unique_grade = cleaned_df['Grade(YDS)'].unique()
unique_grade

array(['5.7', '5.0', '5.7+', '5.8', '5.10b', '5.10a', '5.12a', '5.11d',
       '5.12b/c', '5.12d', '5.10c', '5.11b', '5.11c', '5.6', '5.9',
       '5.12b', '5.10d', '5.11a', '5.11a/b', '5.8+', '5.8-', '5.12-',
       '5.11b/c', '5.12a/b', '5.4', '5.10', '5.10+', '5.13b', '5.13a',
       '5.14a/b', '5.12c', '5.11+', '5.12+', '5.12', '5.10-', '5.13c',
       '5.13+', '5.9-', '5.11c/d', '5.9+', '5.10b/c', '5.11', '5.10c/d',
       '5.13', '5.13d', '5.10a/b', '5.3', '5.5', '5.2', '5.12c/d',
       '5.11-', '5.13-', '5.14-', '5.13b/c', '5.13a/b', '5.15+', '5.14b',
       '5.13c/d', '5.14a', '5.1', '5.14c', '5.14', '5.15a', '5.14c/d',
       '5.14b/c', '5.14+', '5.14d', '5.15', '5.15c/d'], dtype=object)

In [10]:
# Function to categorize grades
def categorize_grade(grade):
    # Extract the numeric part of the grade (ignore letters and +/−)
    base_grade = float(grade.split('.')[1].split('-')[0].split('+')[0].split('a')[0].split('b')[0].split('c')[0].split('d')[0])
    if base_grade <= 9:
        return 'Beginner'
    elif 10 <= base_grade <= 11:
        return 'Intermediate'
    elif 12 <= base_grade <= 13:
        return 'Advanced'
    elif 14 <= base_grade <= 15:
        return 'Pro'
    else:
        return 'Unknown'

cleaned_df['Category'] = cleaned_df['Grade(YDS)'].apply(categorize_grade)

In [11]:
# Exclude rows where 'State' is 'Baja California'
cleaned_df = cleaned_df[cleaned_df['State'] != 'Baja California']

# Reset the index for better readability (optional)
cleaned_df = cleaned_df.reset_index(drop=True)

In [12]:
state_abbreviations = {
    'Iowa': 'IA',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Kansas': 'KS',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Wisconsin': 'WI',
    'Ontario': 'ON',
    'Missouri': 'MO',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'South Dakota': 'SD',
    'Colorado': 'CO',
    'Wyoming': 'WY',
    'California': 'CA',
    'New Mexico': 'NM',
    'Utah': 'UT',
    'Arizona': 'AZ',
    'Idaho': 'ID',
    'Washington': 'WA',
    'Nevada': 'NV',
    'Connecticut': 'CT',
    'Massachusetts': 'MA',
    'Maine': 'ME',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New York': 'NY',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'Vermont': 'VT',
    'Alabama': 'AL',
    'Arkansas': 'AR',
    'Delaware': 'DE',
    'Georgia': 'GA',
    'Kentucky': 'KY',
    'Maryland': 'MD',
    'West Virginia': 'WV',
    'Virginia': 'VA',
    'North Carolina': 'NC',
    'Oklahoma': 'OK',
    'South Carolina': 'SC',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Oregon': 'OR'
}

# Replace state names with abbreviations in the 'State' column
cleaned_df['State'] = cleaned_df['State'].map(state_abbreviations)

In [13]:
# Remove square brackets and quotes from 'Route_Description' and 'Protection_Type'
cleaned_df['Route_Description'] = cleaned_df['Route_Description'].str.strip("[]'\"")
cleaned_df['Protection_Type'] = cleaned_df['Protection_Type'].str.strip("[]'\"")


In [14]:
# For NLP tasks
cleaned_df['Features'] = 'N/A'

In [15]:
cleaned_df.head()


,Route_ID,Route_Name,Route_Type,Sector_ID,Grade(YDS),Route_Description,Protection_Type,Popularity_Votes,State,Route_Quality_Mean,Region/Area,Latitude,Longitude,Average_Rating,Category,Features
0,106956280,Stairway to Heaven,trad,106947227,5.7,Climb the large flake right of Slot Machine to...,"SR, tricams are handy.",10,IA,2.340000,Drive In Wall,-91.5625,42.614,2.600000,Beginner,N/A
1,106956306,Cake Walk,trad,106947239,5.0,Obvious chimney half way up the gully. Severa...,SR,2,IA,1.000000,Cake Walk and Razor Wall,-91.5625,42.614,2.000000,Beginner,N/A
2,106956299,Spinal Tap,trad,106947227,5.7+,Vertical hand crack to the right of Stairway t...,SR,4,IA,1.500000,Drive In Wall,-91.5625,42.614,2.000000,Beginner,N/A
3,106956271,Slot Machine,trad,106947227,5.8,Large off-width/squeeze chimney in the center ...,"SR, there is a chock stone about half way up y...",14,IA,2.387755,Drive In Wall,-91.5625,42.614,2.571429,Beginner,N/A
4,106956212,Lefty and Poncho,trad,106947227,5.10b,Diagonal hand crack that starts on the left si...,"SR, upper head wall not protected.",1,IA,0.000000,Drive In Wall,-91.5625,42.614,2.000000,Intermediate,N/A


## 5. Output into CSV

In [16]:
# Define the output file name
output_file = "cleaned_data.csv"

# Export the DataFrame to a CSV file
cleaned_df.to_csv(output_file, index=False)

# Confirmation message
print(f"DataFrame successfully saved to '{output_file}'")


DataFrame successfully saved to 'cleaned_data.csv'
